In [1]:
import pandas as pd 
import talib

data_path = '../input/'
train = pd.read_pickle(data_path+'/train.pkl')
train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [2]:
train.groupby('Asset_ID').Close.transform(lambda x: x.rolling(window=5).mean())

0                   NaN
1                   NaN
2                   NaN
3                   NaN
4                   NaN
               ...     
24236801     156.809571
24236802    2426.988343
24236803       0.091123
24236804       0.281477
24236805     231.791100
Name: Close, Length: 24236806, dtype: float64

In [3]:
train.groupby('Asset_ID').Close.transform(lambda x: talib.SMA(x, timeperiod=5))

0                   NaN
1                   NaN
2                   NaN
3                   NaN
4                   NaN
               ...     
24236801     156.809571
24236802    2426.988343
24236803       0.091124
24236804       0.281477
24236805     231.791100
Name: Close, Length: 24236806, dtype: float64

In [4]:
import sys
sys.path.append('../src')
from utils import RunningMean


In [5]:
for i in train.Asset_ID.unique():
    a = RunningMean(WIN_SIZE=5)
    a.push(train.loc[train.Asset_ID==i, ['Close', 'Volume']].values)
    print(a.get_mean())


[[2374.59         19.23300519]
 [2372.28666667   24.05025946]
 [2372.06333333   42.67643847]
 ...
 [ 538.775        49.33352893]
 [ 542.5125      183.53930349]
 [ 541.045       582.83068788]]
[[   8.53         78.38      ]
 [   8.5145       71.39      ]
 [   8.4848     1546.82      ]
 ...
 [ 361.858       820.90973643]
 [ 364.1145     1268.05516974]
 [ 363.7575     1000.67694636]]
[[1.38501760e+04 3.15500615e+01]
 [1.38281020e+04 3.10464321e+01]
 [1.38013140e+04 5.50618201e+01]
 ...
 [4.28270200e+04 6.56777342e+01]
 [4.30172771e+04 1.38335477e+02]
 [4.30025050e+04 1.28206820e+02]]
[[7.65760000e+00 6.62671337e+03]
 [7.65670000e+00 3.27747549e+03]
 [7.65120000e+00 5.62355758e+03]
 ...
 [4.17573333e+00 1.82688028e+04]
 [4.19873333e+00 7.79936035e+04]
 [4.19574000e+00 4.10643936e+04]]
[[2.58770000e+01 1.21087310e+02]
 [2.58970000e+01 1.46801900e+00]
 [2.64695000e+01 7.61639216e+01]
 ...
 [4.88267667e+01 5.74742717e+02]
 [4.90782750e+01 3.26924016e+03]
 [4.90681500e+01 1.43435052e+03]]
[[ 7

In [6]:
import numpy as np

def _rsiFunc(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] # cause the diff is 1 shorter

        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi



In [7]:
train.groupby('Asset_ID').Close.transform(lambda x: _rsiFunc(x.values, 5))

0            0.000000
1            9.675481
2            0.000000
3            0.000000
4           86.370262
              ...    
24236801    49.859647
24236802    57.611476
24236803    58.297421
24236804    55.637040
24236805    55.028428
Name: Close, Length: 24236806, dtype: float64

In [8]:
import talib

In [9]:
train.groupby('Asset_ID').Close.transform(lambda x: talib.RSI(x, 5))

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
              ...    
24236801    49.859647
24236802    57.611476
24236803    58.297421
24236804    55.637040
24236805    55.028428
Name: Close, Length: 24236806, dtype: float64